In [ ]:
import pandas as pd
import numpy as np
np.set_printoptions(precision=6, suppress=True)
from sklearn.utils import shuffle

from tqdm import tqdm

import tensorflow as tf
from tensorflow.keras import *
tf.__version__

In [ ]:
tf.config.list_physical_devices('GPU')

In [ ]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        print(e)

In [ ]:
strategy = tf.distribute.MirroredStrategy()

# Hyperparameters

In [ ]:
BEST_PATH = './models/ResNet_1D.h5'
TRAINING_EPOCHS = 200
LEARNING_RATE = 0.02
EPSILON = 1e-06
BATCH_SIZE = 16

# Data loading

In [ ]:
l = np.load('./results/2020_S/fw_dataset.npz', allow_pickle=True)
data_indices = l['data_indices']
input_data = l['input_data']
output_label = l['output_label']
INPUT_MAXS = l['INPUT_MAXS']
INPUT_MINS = l['INPUT_MINS']
OUTPUT_MAX = l['OUTPUT_MAX']
OUTPUT_MIN = l['OUTPUT_MIN']

In [ ]:
input_data = input_data.astype('float32')
output_label = output_label.astype('float32')

In [ ]:
print(input_data.shape)
print(output_label.shape)

In [ ]:
print(INPUT_MAXS)
print(INPUT_MINS)

In [ ]:
print(OUTPUT_MAX)
print(OUTPUT_MIN)

In [ ]:
data_indices, input_data, output_label = shuffle(data_indices, input_data, output_label, random_state=3101)

In [ ]:
N_TRAIN = int(input_data.shape[0]*.8)
train_input = input_data[:N_TRAIN, ...]
train_label = output_label[:N_TRAIN, ...]
val_input = input_data[N_TRAIN:, ...]
val_label = output_label[N_TRAIN:, ...]

In [ ]:
print(f'number of training set: {train_input.shape[0]}')
print(f'number of validation set: {val_input.shape[0]}')

In [ ]:
with strategy.scope():
    train_dataset = tf.data.Dataset.from_tensor_slices((train_input, train_label))
    train_dataset = train_dataset.cache().shuffle(BATCH_SIZE*10).batch(BATCH_SIZE, drop_remainder=True)
    val_dataset = tf.data.Dataset.from_tensor_slices((val_input, val_label))
    val_dataset = val_dataset.cache().shuffle(BATCH_SIZE*10).batch(BATCH_SIZE, drop_remainder=True)

# Model construction

In [ ]:
class ConvBlock(layers.Layer):
    def __init__(self, filters, kernel_size):
        super(ConvBlock, self).__init__()
        self.f = filters
        self.k = kernel_size
        
        self.downsampling = layers.Conv1D(self.f, 1, kernel_initializer='glorot_normal', padding='same')
        self.downbatch = layers.BatchNormalization()
    
        self.conv1 = layers.Conv1D(self.f/4, 1, kernel_initializer='glorot_normal', padding='same')
        self.batch1 = layers.BatchNormalization()
        self.activation1 = layers.Activation(tf.nn.relu)
        
        self.conv2_1 = layers.Conv1D(self.f/4, self.k[0], kernel_initializer='glorot_normal', padding='same')
        self.conv2_2 = layers.Conv1D(self.f/4, self.k[1], kernel_initializer='glorot_normal', padding='same')
        self.conv2_3 = layers.Conv1D(self.f/4, self.k[2], kernel_initializer='glorot_normal', padding='same')
        self.conv2_4 = layers.Conv1D(self.f/4, self.k[3], kernel_initializer='glorot_normal', padding='same')
        self.conv2_5 = layers.Conv1D(self.f/4, self.k[4], kernel_initializer='glorot_normal', padding='same')
        self.batch2 = layers.BatchNormalization()
        self.activation2 = layers.Activation(tf.nn.relu)
        
        self.outconv = layers.Conv1D(self.f, 1, kernel_initializer='glorot_normal', padding='same')
        self.outbatch = layers.BatchNormalization()
        self.outact = layers.Activation(tf.nn.relu)
        
    def call(self, inp):
        
        shortcut = self.downbatch(self.downsampling(inp))
        
        inp = self.activation1(self.batch1(self.conv1(inp)))
        
        inp = tf.concat([self.conv2_1(inp), self.conv2_2(inp), self.conv2_3(inp), self.conv2_4(inp), self.conv2_5(inp)], -1)
        inp = self.activation2(self.batch2(inp))
        
        inp = self.outbatch(self.outconv(inp))
        inp = self.outact(layers.add([shortcut, inp]))
                
        return inp

In [ ]:
class ResNet1D(Model):
    def __init__(self):
        super(ResNet1D, self).__init__()
        self.n = [128, 128, 256, 256, 512] # number of nodes
        self.k = [1, 3, 5, 10, 20] # kernal size
        self.s = 2 # stride (= pooling size)
                
        self.conv_block1_1 = ConvBlock(self.n[0], self.k)
        self.conv_block1_2 = ConvBlock(self.n[0], self.k)
        self.conv_block1_3 = ConvBlock(self.n[0], self.k)
        self.conv_block1_4 = ConvBlock(self.n[0], self.k)
        self.gate1 = layers.Dense(self.n[0], activation=tf.nn.sigmoid)
        self.pool1 = layers.Conv1D(self.n[1], self.s, self.s)

        self.conv_block2_1 = ConvBlock(self.n[1], self.k)
        self.conv_block2_2 = ConvBlock(self.n[1], self.k)
        self.conv_block2_3 = ConvBlock(self.n[1], self.k)
        self.conv_block2_4 = ConvBlock(self.n[1], self.k)
        self.gate2 = layers.Dense(self.n[1], activation=tf.nn.sigmoid)
        self.pool2 = layers.Conv1D(self.n[2], self.s, self.s)
        
        self.conv_block3_1 = ConvBlock(self.n[2], self.k)
        self.conv_block3_2 = ConvBlock(self.n[2], self.k)
        self.conv_block3_3 = ConvBlock(self.n[2], self.k)
        self.conv_block3_4 = ConvBlock(self.n[2], self.k)
        self.gate3 = layers.Dense(self.n[2], activation=tf.nn.sigmoid)
        self.pool3 = layers.Conv1D(self.n[3], self.s, self.s)
        
        self.conv_block4_1 = ConvBlock(self.n[3], self.k)
        self.conv_block4_2 = ConvBlock(self.n[3], self.k)
        self.conv_block4_3 = ConvBlock(self.n[3], self.k)
        self.conv_block4_4 = ConvBlock(self.n[3], self.k)
        self.conv_block4_5 = ConvBlock(self.n[3], self.k)
        self.gate4 = layers.Dense(self.n[3], activation=tf.nn.sigmoid)
        self.pool4 = layers.Conv1D(self.n[4], self.s, self.s)
        
        self.conv_block5_1 = ConvBlock(self.n[4], self.k)
        self.conv_block5_2 = ConvBlock(self.n[4], self.k)
        self.conv_block5_3 = ConvBlock(self.n[4], self.k)
        self.conv_block5_4 = ConvBlock(self.n[4], self.k)
        self.conv_block5_5 = ConvBlock(self.n[4], self.k)
        self.conv_block5_6 = ConvBlock(self.n[4], self.k)
        self.gate5 = layers.Dense(self.n[4], activation=tf.nn.sigmoid)
        
#         self.output_conv = layers.Conv1D(1, 1, activation='sigmoid')
        self.outgate = layers.Dense(1, activation=tf.nn.sigmoid)
        
    def call(self, inp):
        
        conv1 = self.conv_block1_1(inp)
        conv1 = self.conv_block1_2(conv1)
        conv1 = self.conv_block1_3(conv1)
        conv1 = self.conv_block1_4(conv1)
        conv1 = self.gate1(conv1)
        conv1 = self.pool1(conv1)
        
        conv2 = self.conv_block2_1(conv1)
        conv2 = self.conv_block2_2(conv2)
        conv2 = self.conv_block2_3(conv2)
        conv2 = self.conv_block2_4(conv2)
        conv2 = self.gate2(conv2)
        conv2 = self.pool2(conv2)
        
        conv3 = self.conv_block3_1(conv2)
        conv3 = self.conv_block3_2(conv3)
        conv3 = self.conv_block3_3(conv3)
        conv3 = self.conv_block3_4(conv3)
        conv3 = self.gate3(conv3)
        conv3 = self.pool3(conv3)
        
        conv4 = self.conv_block4_1(conv3)
        conv4 = self.conv_block4_2(conv4)
        conv4 = self.conv_block4_3(conv4)
        conv4 = self.conv_block4_4(conv4)
        conv4 = self.conv_block4_5(conv4)
        conv4 = self.gate4(conv4)
        conv4 = self.pool4(conv4)

        conv5 = self.conv_block5_1(conv4)
        conv5 = self.conv_block5_2(conv5)
        conv5 = self.conv_block5_3(conv5)
        conv5 = self.conv_block5_4(conv5)
        conv5 = self.conv_block5_5(conv5)
        conv5 = self.conv_block5_6(conv5)
        conv5 = self.gate5(conv5)
        
        return self.outgate(conv5)

In [ ]:
callbacks = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss', factor=.5, patience=2, verbose=0, mode='min',
    min_delta=0.0001, cooldown=0, min_lr=0)

save = tf.keras.callbacks.ModelCheckpoint(
    BEST_PATH, monitor='val_loss', verbose=0,
    save_best_only=True, save_weights_only=True, mode='min', save_freq='epoch')

early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0.0001, patience=20) 

In [ ]:
with strategy.scope():
    opt = tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE, epsilon=EPSILON)
    model = ResNet1D()
    model.compile(optimizer=opt, loss='mse')
    model.fit(train_dataset, epochs=TRAINING_EPOCHS, validation_data=val_dataset,
                  verbose=1, callbacks=[callbacks, save, early_stop]) 

In [ ]:
model.load_weights(BEST_PATH)

In [ ]:
model.evaluate(val_dataset)

0.019411545246839523